In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.model_selection import KFold

# Part I 

## Linear Regression

### Matrix building functions

In [ ]:
def matrix_design(x, ks):
    x = np.asarray(x).reshape(-1)
    ks = np.asarray(ks)
    n = x.shape[0]
    K = ks.shape[0]
    X = np.zeros((n, K))
    for j in range(K):
        X[:, j] = x ** ks[j]
    return X

def sin_matrix(x, ks):
    x = np.asarray(x).reshape(-1)
    ks = np.asarray(ks)
    n = x.shape[0]
    K = ks.shape[0]
    X = np.zeros((n, K))
    for j in range(K):
        X[:, j] = np.sin(ks[j] * np.pi * x)
    return X

### Q1. Polynomial bases

In [ ]:
# Data
x = np.array([1, 2, 3, 4], dtype=float).reshape(-1, 1)
y = np.array([3, 2, 0, 5], dtype=float).reshape(-1, 1)

xx = np.linspace(0.0, 5.0, 200).reshape(-1, 1)

# Colors and line styles for each fit
styles = ["b-", "r--", "g-.", "m:"]
order = np.arange(1, 5)  # k = 1,2,3,4

formulas = [None] * len(order)
sses = np.zeros(len(order))

plt.figure()
plt.plot(x, y, "ko", markerfacecolor="k", label="Data")

for ki, k in enumerate(order):
    ks = np.arange(0, k)  # [0], [0 1], [0 1 2], [0 1 2 3]

    X = matrix_design(x, ks)
    # Least squares solve
    w, *_ = np.linalg.lstsq(X, y, rcond=None)

    X_fit = matrix_design(xx, ks)
    y_fit = X_fit @ w
    ytrain_fit = X @ w

    sses[ki] = np.sum((y - ytrain_fit) ** 2)

    plt.plot(xx, y_fit, styles[ki], linewidth=2, label=f"k={k}")

    # Build explicit formula string
    terms = []
    for d in range(k):
        coeff = w[d, 0]
        if abs(coeff) > 1e-12:
            if ks[d] == 0:
                terms.append(f"{coeff:.3g}")
            elif ks[d] == 1:
                terms.append(f"{coeff:.3g}x")
            else:
                terms.append(f"{coeff:.3g}x^{ks[d]}")
    formulas[ki] = "y = " + " + ".join(terms) if terms else "y = 0"

plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="upper left")
plt.title("Polynomial Regression Fits: k=1,2,3,4")
plt.show()

# Display formulas
print("\nPolynomial formulas:")
for k, fml in zip(order, formulas):
    print(f"k={k}: {fml}")

# Display SSE vector
print("Sum of Squared Errors (SSE):")
print(sses)

### Q2. (a) Polynomial bases with sine function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sigma = 0.07  # noise standard deviation
n = 30        # number of sample points

# Sample xi uniformly from [0,1]
x_data = np.random.rand(n, 1)

def g_sigma(x):
    x = np.asarray(x)
    return np.sin(2 * np.pi * x) ** 2 + sigma * np.random.randn(*x.shape)

# Noisy outputs
y_data = g_sigma(x_data)

# For plotting the underlying function (smooth curve):
xx = np.linspace(0.0, 1.0, 300).reshape(-1, 1)
true_func = np.sin(2 * np.pi * xx) ** 2

plt.figure()
plt.plot(xx, true_func, "b-", linewidth=2, label=r"sin^2(2πx)")
plt.plot(x_data, y_data, "ro", markerfacecolor="r", label="Noisy Data Points")
plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="upper left")
plt.title(r"sin^2(2πx) and Noisy Data Points")
plt.show()

### (b) training error

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sigma = 0.07  # noise standard deviation
ntrain = 30   # number of sample points

x_train = np.random.rand(ntrain, 1)
mtrain = x_train.shape[0]

def g_sigma(x):
    x = np.asarray(x)
    return np.sin(2 * np.pi * x) ** 2 + sigma * np.random.randn(*x.shape)

y_train = g_sigma(x_train)

xx = np.linspace(0.0, 1.0, 400).reshape(-1, 1)
true_func = np.sin(2 * np.pi * xx) ** 2

styles = ["b-", "r--", "g-.", "m:", "c-."]
bases = np.array([2, 5, 10, 14, 18])

tek = np.zeros_like(bases, dtype=float)
ssestrain = np.zeros_like(bases, dtype=float)

plt.figure()
plt.plot(xx, true_func, "k-", linewidth=1.5, label="True function")
plt.plot(x_train, y_train, "ro", markerfacecolor="r", label="Noisy data")

for ki, k in enumerate(bases):
    ks = np.arange(0, k)  # basis exponents

    Xtrain = matrix_design(x_train, ks)
    w, *_ = np.linalg.lstsq(Xtrain, y_train, rcond=None)

    X_fit = matrix_design(xx, ks)
    y_fit = X_fit @ w
    ytrain_fit = Xtrain @ w

    ssestrain[ki] = np.sum((y_train - ytrain_fit) ** 2)
    tek[ki] = ssestrain[ki] / mtrain

    plt.plot(xx, y_fit, styles[ki], linewidth=2,
             label=f"Poly degree {k-1}")

plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="upper left")
plt.title("Polynomial Regression Fits: k=2,5,10,14,18")
plt.ylim([-1, 2])
plt.show()

plt.figure()
plt.plot(bases, np.log(tek), "bo-", linewidth=2, markerfacecolor="b")
plt.xlabel("Polynomial Dimension k")
plt.ylabel("ln(Training MSE)")
plt.title("Natural Log of Training Error vs Polynomial Dimension")
plt.grid(True)
plt.show()

### (c) test error

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
sigma = 0.07  # noise standard deviation

def g_sigma(x):
    x = np.asarray(x)
    return np.sin(2 * np.pi * x) ** 2 + sigma * np.random.randn(*x.shape)

xx = np.linspace(0.0, 1.0, 400).reshape(-1, 1)  # for smooth plotting
true_func = np.sin(2 * np.pi * xx) ** 2

styles = ["b-", "r--", "g-.", "m:", "c-."]
bases = np.array([2, 5, 10, 14, 18])

# -------------------------------
ntrain = 30
x_train = np.random.rand(ntrain, 1)
mtrain = x_train.shape[0]
y_train = g_sigma(x_train)

tek = np.zeros_like(bases, dtype=float)
ssestrain = np.zeros_like(bases, dtype=float)

# -------------------------------
ntest = 1000
x_test = np.random.rand(ntest, 1)
mtest = x_test.shape[0]
y_test = g_sigma(x_test)
test_true = np.sin(2 * np.pi * x_test) ** 2

tseks = np.zeros_like(bases, dtype=float)
ssestest = np.zeros_like(bases, dtype=float)

plt.figure()
plt.plot(xx, true_func, "k-", linewidth=2, label="True function")
plt.plot(x_test, y_test, "go", markerfacecolor="g", linestyle="",
         label="Test (noisy)")
plt.plot(x_test, test_true, "rx", markersize=6, linestyle="",
         label="Test (true)")

for ki, k in enumerate(bases):
    ks = np.arange(0, k)

    Xtrain = matrix_design(x_train, ks)
    w, *_ = np.linalg.lstsq(Xtrain, y_train, rcond=None)

    X_fit = matrix_design(xx, ks)
    y_fit = X_fit @ w
    plt.plot(xx, y_fit, styles[ki], linewidth=2,
             label=f"Poly degree {k-1}")

    ytrain_fit = Xtrain @ w
    ssestrain[ki] = np.sum((y_train - ytrain_fit) ** 2)
    tek[ki] = ssestrain[ki] / mtrain

    Xtest = matrix_design(x_test, ks)
    ytest_fit = Xtest @ w
    ssestest[ki] = np.sum((y_test - ytest_fit) ** 2)
    tseks[ki] = ssestest[ki] / mtest

plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="upper left")
plt.title("True Func, Test Data (Noisy + True), Fitted Polynomials")
plt.show()

plt.figure()
plt.plot(bases, np.log(tek), "bo-", linewidth=2, markerfacecolor="b", label="Training error")
plt.plot(bases, np.log(tseks), "ro-", linewidth=2, markerfacecolor="r", label="Test error")
plt.xlabel("Polynomial Dimension k")
plt.ylabel("ln(MSE)")
plt.title("Training (blue) and Test (red) Error vs Polynomial Dimension")
plt.legend(loc="upper left")
plt.grid(True)
plt.show()

### (d) log of average MSE

In [ ]:
mse_test_all = np.zeros((num_runs, len(bases)))

for run in range(num_runs):
    # Random training and test sets
    x_train = np.random.rand(ntrain, 1)
    y_train = np.sin(2 * np.pi * x_train) ** 2 + 0.07 * np.random.randn(ntrain, 1)

    x_test = np.random.rand(ntest, 1)
    y_test = np.sin(2 * np.pi * x_test) ** 2 + 0.07 * np.random.randn(ntest, 1)

    for ki, k in enumerate(bases):
        ks = np.arange(0, k)  # 0..k-1
        Xtrain = matrix_design(x_train, ks)
        wtrain, *_ = np.linalg.lstsq(Xtrain, y_train, rcond=None)

        Xtest = matrix_design(x_test, ks)
        ytest_pred = Xtest @ wtrain

        mse = np.mean((y_test - ytest_pred) ** 2)
        mse_test_all[run, ki] = mse

avg_mse = mse_test_all.mean(axis=0)  # shape (len(bases),)

plt.figure()
plt.plot(bases, np.log(avg_mse), "ro-", linewidth=2, markerfacecolor="r")
plt.xlabel("Polynomial Dimension k")
plt.ylabel("ln(avg Test MSE over {} runs)".format(num_runs))
plt.title("Log Average Test Error vs Polynomial Dimension")
plt.grid(True)
plt.show()

### Q3. Sine basis

In [ ]:
sigma = 0.07

def g_sigma(x):
    x = np.asarray(x)
    return np.sin(2 * np.pi * x) ** 2 + sigma * np.random.randn(*x.shape)

xx = np.linspace(0.0, 1.0, 400).reshape(-1, 1)
true_func = np.sin(2 * np.pi * xx) ** 2

bases = np.array([2, 5, 10, 14, 18])
runs = 100
ntest = 1000
ntrain = 30
max_k = len(bases)

mse_all = np.zeros((runs, max_k))
mse_train = np.zeros((runs, max_k))

# Visualisation test/train sets
x_test_vis = np.random.rand(ntest, 1)
y_test_vis = g_sigma(x_test_vis)
test_true_vis = np.sin(2 * np.pi * x_test_vis) ** 2

x_train_vis = np.random.rand(ntrain, 1)
y_train_vis = g_sigma(x_train_vis)

plt.figure()
plt.plot(xx, true_func, "k-", linewidth=2, label="True function")
plt.plot(x_test_vis, y_test_vis, "go", markerfacecolor="g",
         linestyle="", label="Test (noisy)")
plt.plot(x_test_vis, test_true_vis, "rx", markersize=6,
         linestyle="", label="Test (true)")
plt.plot(x_train_vis, y_train_vis, "bo", markerfacecolor="b",
         linestyle="", label="Training data")
styles = ["b-", "r--", "g-.", "m:", "c-."]
plt.legend(loc="upper left")
plt.title("True Functions of Sine Basis")
plt.show()

plt.figure()
for ki, k in enumerate(bases):
    ks = np.arange(1, k + 1)
    Xtrain_vis = sin_matrix(x_train_vis, ks)
    wvis, *_ = np.linalg.lstsq(Xtrain_vis, y_train_vis, rcond=None)

    Xvis = sin_matrix(xx, ks)
    y_vis = Xvis @ wvis

    plt.plot(xx, y_vis, styles[ki], linewidth=2,
             label=f"Sine basis k={k}")
plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="upper left")
plt.title("Sine Basis Fits: k=2,5,10,14,18")
plt.show()

# Monte Carlo
for run in range(runs):
    x_train = np.random.rand(ntrain, 1)
    y_train = g_sigma(x_train)
    x_test = np.random.rand(ntest, 1)
    y_test = g_sigma(x_test)

    for ki, k in enumerate(bases):
        ks = np.arange(1, k + 1)

        Xtrain = sin_matrix(x_train, ks)
        w, *_ = np.linalg.lstsq(Xtrain, y_train, rcond=None)
        ytrain_fit = Xtrain @ w
        mse_train[run, ki] = np.mean((y_train - ytrain_fit) ** 2)

        Xtest = sin_matrix(x_test, ks)
        ytest_fit = Xtest @ w
        mse_all[run, ki] = np.mean((y_test - ytest_fit) ** 2)

avg_train = mse_train.mean(axis=0)
avg_mse = mse_all.mean(axis=0)

plt.figure()
plt.plot(bases, np.log(avg_train), "bo-", linewidth=2,
         markerfacecolor="b", label="Train MSE")
plt.plot(bases, np.log(avg_mse), "ro-", linewidth=2,
         markerfacecolor="r", label="Test MSE")
plt.xlabel("Basis Dimension k")
plt.ylabel("ln(MSE)")
plt.title("Train (blue) and Test (red) Error vs Sine Basis Dimension")
plt.legend(loc="upper left")
plt.grid(True)
plt.show()

## Filtered Boston housing and kernels

### Q4 Baseline versus full linear regression

In [ ]:
# Hyperparameter grids
gamma_vals = 2.0 ** np.arange(-40, -25)        # 15 gamma values
sigma_vals = 2.0 ** np.arange(7, 13.5, 0.5)    # 13 sigma values

# Load data
T = pd.read_csv("/datasets/t1cw-data/Boston-filtered.txt", sep=None, engine="python")
X = T.iloc[:, :-1].to_numpy()
y = T.iloc[:, -1].to_numpy()
n, d = X.shape
attribute_names = T.columns[:-1].to_list()

runs = 20

mse_krr_train = np.zeros(runs)
mse_krr_test = np.zeros(runs)
best_gamma_all = np.zeros(runs)
best_sigma_all = np.zeros(runs)

mse_naive_train = np.zeros(runs)
mse_naive_test = np.zeros(runs)

mse_linear_train = np.zeros((runs, d))
mse_linear_test = np.zeros((runs, d))

mse_linear_all_train = np.zeros(runs)
mse_linear_all_test = np.zeros(runs)

for r in range(runs):
    idx = np.random.permutation(n)
    ntrain = int(round(2 * n / 3))
    train_idx = idx[:ntrain]
    test_idx = idx[ntrain:]

    X_train = X[train_idx, :]
    y_train = y[train_idx]
    X_test = X[test_idx, :]
    y_test = y[test_idx]


### (a) Naive regression

In [ ]:
    w_const = y_train.mean()
    ytrain_fit = np.full_like(y_train, w_const, dtype=float)
    ytest_fit = np.full_like(y_test, w_const, dtype=float)
    mse_naive_train[r] = np.mean((y_train - ytrain_fit) ** 2)
    mse_naive_test[r] = np.mean((y_test - ytest_fit) ** 2)

print(f"{'Naive Regression':40s} "
      f"{naive_train_mean:6.2f} ± {naive_train_std:4.2f} "
      f"{naive_test_mean:6.2f} ± {naive_test_std:4.2f}")


### (c) Single‑attribute linear + bias

In [ ]:
for j in range(d):
        xj_train = X_train[:, j:j+1]
        xj_test = X_test[:, j:j+1]

        Xj_train = np.hstack([xj_train, np.ones((ntrain, 1))])
        Xj_test = np.hstack([xj_test, np.ones((xj_test.shape[0], 1))])

        wj, *_ = np.linalg.lstsq(Xj_train, y_train, rcond=None)
        ytrain_fit = Xj_train @ wj
        ytest_fit = Xj_test @ wj

        mse_linear_train[r, j] = np.mean((y_train - ytrain_fit) ** 2)
        mse_linear_test[r, j] = np.mean((y_test - ytest_fit) ** 2)


for j, name in enumerate(attribute_names):
    label = f"Linear Regression ({name})"
    print(f"{label:40s} "
          f"{linear_train_mean[j]:6.2f} ± {linear_train_std[j]:4.2f} "
          f"{linear_test_mean[j]:6.2f} ± {linear_test_std[j]:4.2f}")

### (d) Full linear regression with bias

In [ ]:
for r in range(runs):
    idx = np.random.permutation(n)
    ntrain = int(round(2 * n / 3))
    train_idx = idx[:ntrain]
    test_idx = idx[ntrain:]

    Xaug = np.hstack([X, np.ones((n, 1))])
    Xtrain_aug = Xaug[train_idx, :]
    Xtest_aug = Xaug[test_idx, :]

    w_full, *_ = np.linalg.lstsq(Xtrain_aug, y_train, rcond=None)
    ytrain_fit = Xtrain_aug @ w_full
    ytest_fit = Xtest_aug @ w_full

    mse_linear_all_train[r] = np.mean((y_train - ytrain_fit) ** 2)
    mse_linear_all_test[r] = np.mean((y_test - ytest_fit) ** 2)

print(f"{'Linear Regression (all attributes)':40s} "
      f"{linear_all_train_mean:6.2f} ± {linear_all_train_std:4.2f} "
      f"{linear_all_test_mean:6.2f} ± {linear_all_test_std:4.2f}")

## Q5. Kernelised ridge regression

### Kernel function

In [ ]:
def kernel_func(X1, X2, sigma):
    X1 = np.asarray(X1)
    X2 = np.asarray(X2)
    D2 = cdist(X1, X2, metric="euclidean") ** 2
    K = np.exp(-D2 / (2.0 * sigma ** 2))
    return K

In [ ]:
# Hyperparameter grids
gamma_vals = 2.0 ** np.arange(-40, -25)   # 15 gamma values
sigma_vals = 2.0 ** np.arange(7, 13.5, 0.5)  # 13 sigma values

runs = 20
train_mse_all = np.zeros(runs)
test_mse_all = np.zeros(runs)
best_gamma_all = np.zeros(runs)
best_sigma_all = np.zeros(runs)

# For plotting CV error for the first run
cv_error_grid = None

# Load data
T = pd.read_csv("/datasets/t1cw-data/Boston-filtered.txt", sep=None, engine="python")
X = T.iloc[:, :-1].to_numpy()
y = T.iloc[:, -1].to_numpy()
n = y.shape[0]

for run in range(runs):
    idx = np.random.permutation(n)
    ntrain = int(round(2 * n / 3))
    train_idx = idx[:ntrain]
    test_idx = idx[ntrain:]

    X_train = X[train_idx, :]
    y_train = y[train_idx]
    X_test = X[test_idx, :]
    y_test = y[test_idx]

### (a) kFold CV

In [ ]:
    # --- Cross‑validation over gamma/sigma grid ---
    kf = KFold(n_splits=5, shuffle=True, random_state=run)
    mean_cv_error = np.zeros((len(gamma_vals), len(sigma_vals)))

    for gi, gamma in enumerate(gamma_vals):
        for si, sigma in enumerate(sigma_vals):
            mse_fold = []

            for train_fold_idx, val_fold_idx in kf.split(X_train):
                Xtr = X_train[train_fold_idx]
                ytr = y_train[train_fold_idx]
                Xval = X_train[val_fold_idx]
                yval = y_train[val_fold_idx]

                Ktr = kernel_func(Xtr, Xtr, sigma)
                Kval = kernel_func(Xval, Xtr, sigma)

                # (K + gamma * n * I)^{-1} y
                n_tr = ytr.shape[0]
                alpha = np.linalg.solve(
                    Ktr + gamma * n_tr * np.eye(Ktr.shape[0]),
                    ytr,
                )
                yval_fit = Kval @ alpha
                mse_fold.append(np.mean((yval - yval_fit) ** 2))

            mean_cv_error[gi, si] = np.mean(mse_fold)

### (b) Cross-validation error

In [ ]:
    if run == 0:
        cv_error_grid = mean_cv_error.copy()
        
plt.figure()
plt.imshow(np.log10(cv_error_grid),
           aspect="auto",
           origin="lower")
plt.colorbar(label="log10(CV MSE)")
plt.xlabel("sigma index")
plt.ylabel("gamma index")
plt.title("Log10 Cross‑Validation Error Surface")

xticks = np.arange(len(sigma_vals))
yticks = np.arange(len(gamma_vals))
plt.xticks(xticks, [f"{np.log2(s):.1f}" for s in sigma_vals], rotation=45)
plt.yticks(yticks, [f"{int(np.log2(g))}" for g in gamma_vals])
plt.tight_layout()
plt.show()

### (c) Best gamma, sigma and final KRR fit 

In [ ]:
    ind = np.argmin(mean_cv_error)
    best_g_idx, best_s_idx = np.unravel_index(ind, mean_cv_error.shape)
    best_gamma = gamma_vals[best_g_idx]
    best_sigma = sigma_vals[best_s_idx]
    best_gamma_all[r] = best_gamma
    best_sigma_all[r] = best_sigma

    Ktr = kernel_func(X_train, X_train, best_sigma)
    n_tr = y_train.shape[0]
    alpha = np.linalg.solve(
        Ktr + best_gamma * n_tr * np.eye(Ktr.shape[0]),
        y_train,
    )
    Ktest = kernel_func(X_test, X_train, best_sigma)
    y_train_fit = Ktr @ alpha
    y_test_fit = Ktest @ alpha

    mse_krr_train[r] = np.mean((y_train - y_train_fit) ** 2)
    mse_krr_test[r] = np.mean((y_test - y_test_fit) ** 2)

### (d) Aggregate statistics

In [ ]:
naive_train_mean = mse_naive_train.mean()
naive_test_mean = mse_naive_test.mean()
naive_train_std = mse_naive_train.std()
naive_test_std = mse_naive_test.std()

linear_train_mean = mse_linear_train.mean(axis=0)
linear_test_mean = mse_linear_test.mean(axis=0)
linear_train_std = mse_linear_train.std(axis=0)
linear_test_std = mse_linear_test.std(axis=0)

linear_all_train_mean = mse_linear_all_train.mean()
linear_all_test_mean = mse_linear_all_test.mean()
linear_all_train_std = mse_linear_all_train.std()
linear_all_test_std = mse_linear_all_test.std()

krr_train_mean = mse_krr_train.mean()
krr_test_mean = mse_krr_test.mean()
krr_train_std = mse_krr_train.std()
krr_test_std = mse_krr_test.std()

print(f"{'Method':40s} {'MSE train':>20s} {'MSE test':>20s}")
print("-" * 80)

print(f"{'Naive Regression':40s} "
      f"{naive_train_mean:6.2f} ± {naive_train_std:4.2f} "
      f"{naive_test_mean:6.2f} ± {naive_test_std:4.2f}")

for j, name in enumerate(attribute_names):
    label = f"Linear Regression ({name})"
    print(f"{label:40s} "
          f"{linear_train_mean[j]:6.2f} ± {linear_train_std[j]:4.2f} "
          f"{linear_test_mean[j]:6.2f} ± {linear_test_std[j]:4.2f}")

print(f"{'Linear Regression (all attributes)':40s} "
      f"{linear_all_train_mean:6.2f} ± {linear_all_train_std:4.2f} "
      f"{linear_all_test_mean:6.2f} ± {linear_all_test_std:4.2f}")

print(f"{'Kernel Ridge Regression':40s} "
      f"{krr_train_mean:6.2f} ± {krr_train_std:4.2f} "
      f"{krr_test_mean:6.2f} ± {krr_test_std:4.2f}")

# Part II

## k-NN

In [ ]:
# helper functions

def sample_pH(N):
    centers = np.random.rand(N, 2)
    labels = np.random.randint(0, 2, size=N)
    return centers, labels


def voted_center(x, centers, labels, v):
    x = np.asarray(x).reshape(1, -1)
    centers = np.asarray(centers)
    labels = np.asarray(labels).reshape(-1)
    dists = np.sqrt(np.sum((centers - x) ** 2, axis=1))
    idx = np.argsort(dists)
    nearest_labels = labels[idx[:v]]
    count0 = np.sum(nearest_labels == 0)
    count1 = np.sum(nearest_labels == 1)
    if count0 > count1:
        return 0
    elif count1 > count0:
        return 1
    else:
        return np.nan

### Q6. Visualisation of a voted‑center hypothesis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 100       # number of centers
v = 3         # number of neighbors in the vote
gridRes = 200 # grid resolution

centers, labels = sample_pH(N)

xg, yg = np.meshgrid(
    np.linspace(0.0, 1.0, gridRes),
    np.linspace(0.0, 1.0, gridRes),
)
hg = np.full_like(xg, fill_value=np.nan, dtype=float)

for i in range(xg.size):
    x = np.array([xg.flat[i], yg.flat[i]])

    dists = np.sqrt(np.sum((centers - x) ** 2, axis=1))
    idx = np.argsort(dists)
    nearest_labels = labels[idx[:v]]

    c0 = np.sum(nearest_labels == 0)
    c1 = np.sum(nearest_labels == 1)
    if c0 > c1:
        hg.flat[i] = 0
    elif c1 > c0:
        hg.flat[i] = 1
    else:
        hg.flat[i] = np.nan

hg_plot = hg + 1  # 0→1, 1→2
hg_plot[np.isnan(hg)] = 3  # ambiguous→3

plt.figure()
plt.imshow(
    hg_plot,
    extent=(0, 1, 0, 1),
    origin="lower",
    aspect="equal",
)
cmap = np.array([
    [255, 255, 255],  # class 0 -> white
    [64, 224, 208],   # class 1 -> turquoise
    [128, 128, 128],  # ambiguous -> gray
]) / 255.0
plt.colormaps.register(cmap=plt.matplotlib.colors.ListedColormap(cmap))
plt.set_cmap("h_{S,v} sampled from p_H")
plt.clim(1, 3)
plt.colorbar()

# Plot centers
plt.plot(centers[labels == 0, 0], centers[labels == 0, 1],
         "ro", markerfacecolor="r", markersize=6, label="Class 0")
plt.plot(centers[labels == 1, 0], centers[labels == 1, 1],
         "bo", markerfacecolor="b", markersize=6, label="Class 1")

plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.title("Visualisation of a voted-center hypothesis h_{S,v} sampled from p_H")
plt.show()

### Q7. Protocol A (estimated generalization error as a function of k)

In [ ]:
K_max = 49
runs = 100
n_train = 4000
n_test = 1000

errors = np.zeros((K_max, runs))

for k in range(1, K_max + 1):
    for run in range(runs):
        centers, labels = sample_pH(100)

        X_train = np.random.rand(n_train, 2)
        Y_train = np.zeros(n_train, dtype=int)
        for i in range(n_train):
            if np.random.rand() < 0.8:
                y_val = voted_center(X_train[i, :], centers, labels, 3)
                if np.isnan(y_val):
                    y_val = np.random.randint(0, 2)
                Y_train[i] = int(y_val)
            else:
                Y_train[i] = np.random.randint(0, 2)

        X_test = np.random.rand(n_test, 2)
        Y_test = np.zeros(n_test, dtype=int)
        for i in range(n_test):
            if np.random.rand() < 0.8:
                y_val = voted_center(X_test[i, :], centers, labels, 3)
                if np.isnan(y_val):
                    y_val = np.random.randint(0, 2)
                Y_test[i] = int(y_val)
            else:
                Y_test[i] = np.random.randint(0, 2)

        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, Y_train)
        pred = knn.predict(X_test)

        errors[k - 1, run] = np.mean(pred != Y_test)

mean_errors = errors.mean(axis=1)

plt.figure()
plt.plot(np.arange(1, K_max + 1), mean_errors, "-o")
plt.xlabel("k")
plt.ylabel("Estimated Generalisation Error")
plt.title("k‑NN Generalisation Error vs. k (Protocol A)")
plt.grid(True)
plt.show()

### Q8. Protocol B (optimal k as a function of m)

In [ ]:
M_vals = np.array([100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000])
K_max = 49
runs = 100
n_test = 1000

mean_optimal = np.zeros(len(M_vals))

for m_idx, m in enumerate(M_vals):
    optimal = np.zeros(runs, dtype=int)

    for run in range(runs):
        centers, labels = sample_pH(100)

        X_train = np.random.rand(m, 2)
        Y_train = np.zeros(m, dtype=int)
        for i in range(m):
            y_val = voted_center(X_train[i, :], centers, labels, 3)
            if np.isnan(y_val):
                y_val = np.random.randint(0, 2)

            if np.random.rand() < 0.8:
                Y_train[i] = int(y_val)
            else:
                Y_train[i] = np.random.randint(0, 2)

        X_test = np.random.rand(n_test, 2)
        Y_test = np.zeros(n_test, dtype=int)
        for i in range(n_test):
            y_val = voted_center(X_test[i, :], centers, labels, 3)
            if np.isnan(y_val):
                y_val = np.random.randint(0, 2)

            if np.random.rand() < 0.8:
                Y_test[i] = int(y_val)
            else:
                Y_test[i] = np.random.randint(0, 2)

        errs = np.zeros(K_max)
        for k in range(1, K_max + 1):
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, Y_train)
            pred = knn.predict(X_test)
            errs[k - 1] = np.mean(pred != Y_test)

        best_k = np.argmin(errs) + 1
        optimal[run] = best_k

    mean_optimal[m_idx] = optimal.mean()

plt.figure()
plt.plot(M_vals, mean_optimal, "-o")
plt.xlabel("Training set size m")
plt.ylabel("Mean estimated optimal k")
plt.title("Protocol B: m vs mean optimal k")
plt.grid(True)
plt.show()

# Part III

## Q11. Gaussian elimination mod 2 (whack‑a‑mole)

In [ ]:
A = np.array([
    [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1],
], dtype=int)

b = np.array([0, 1, 0, 0, 1, 1, 1, 0,
              0, 0, 0, 1, 1, 1, 0, 1], dtype=int).reshape(-1, 1)

Ab = np.hstack([A, b])
m, n_tot = Ab.shape
n = n_tot - 1  # number of variables

# Forward elimination mod 2
row = 0
for col in range(n):
    # pivot: find row with a 1 in this column at or below current row
    pivot_rows = np.where(Ab[row:, col] % 2 != 0)[0]
    if pivot_rows.size == 0:
        continue
    p = row + pivot_rows[0]

    if p != row:
        Ab[[row, p], :] = Ab[[p, row], :]

    # eliminate below
    for r in range(row + 1, m):
        if Ab[r, col] % 2 != 0:
            Ab[r, :] = (Ab[r, :] + Ab[row, :]) % 2

    row += 1
    if row == m:
        break

# Back substitution mod 2
x = np.zeros((n, 1), dtype=int)

for i in range(m - 1, -1, -1):
    pivot_cols = np.where(Ab[i, :n] % 2 != 0)[0]
    if pivot_cols.size == 0:
        continue
    j = pivot_cols[0]
    rhs = (Ab[i, n] - (Ab[i, j+1:n] @ x[j+1:n, 0]) % 2) % 2
    x[j, 0] = rhs

print("Solution vector x (which moles to press):")
print(x.reshape(-1))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cb182644-878e-48cb-992b-68a78a5afe3d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>